In [2]:
from datasets import load_dataset

dataset = load_dataset("searle-j/kote")
print(dataset)

/data/ephemeral/home/level4-nlp-finalproject-hackathon-nlp-03-lv3/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 40000 examples [00:01, 23842.40 examples/s]
Generating test split: 5000 examples [00:00, 24957.87 examples/s]
Generating validation split: 5000 examples [00:00, 24974.75 examples/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 5000
    })
})


In [5]:
import os

# 데이터셋이 있는 폴더 생성
os.makedirs("data", exist_ok=True)

# 각 데이터셋 파트를 CSV로 저장
for split in dataset.keys():
    dataset[split].to_csv(f"data/{split}.csv", index=False)

Creating CSV from Arrow format: 100%|██████████| 5/5 [00:00<00:00, 22.49ba/s]


### 43개 감정을 긍정/부정/중립으로 세 분류
- 긍정: 환영/호의, 감동/감탄, 고마움, 존경, 기대감, 비장함, 뿌듯함, 편안/쾌적, 신기함/관심, 아껴주는, 부끄러움, 즐거움/신남, 깨달음, 흐뭇함(귀여움/예쁨), 행복, 기쁨, 안심/신뢰
- 부정: 화남/분노, 안타까움/실망, 의심/불신, 한심함, 역겨움/징그러움, 짜증, 어이없음, 증오/혐오, 경악, 재미없음, 불평/불만
- 중립: 슬픔, 우쭐댐/무시함, 공포/무서움, 절망, 패배/자기혐오, 귀찮음, 힘듦/지침, 죄책감, 당황/난처, 부담/안내킴, 서러움, 불쌍함/연민, 놀람, 불안/걱정, 지긋지긋

In [17]:
import pandas as pd

train = pd.read_csv('data/train.csv')
valid = pd.read_csv('data/validation.csv')
test = pd.read_csv('data/test.csv')

data = pd.concat([train, valid, test], ignore_index=True)

In [19]:
# 라벨 데이터
LABELS = [
    '불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움', '슬픔', '화남/분노', '존경', '기대감', '우쭐댐/무시함', 
    '안타까움/실망', '비장함', '의심/불신', '뿌듯함', '편안/쾌적', '신기함/관심', '아껴주는', '부끄러움', '공포/무서움', 
    '절망', '한심함', '역겨움/징그러움', '짜증', '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침', 
    '즐거움/신남', '깨달음', '죄책감', '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처', '경악', '부담/안내킴', 
    '서러움', '재미없음', '불쌍함/연민', '놀람', '행복', '불안/걱정', '기쁨', '안심/신뢰'
]

# 라벨 카테고리 매핑
positive_labels = {'환영/호의', '감동/감탄', '고마움', '존경', '기대감', '비장함', '뿌듯함', '편안/쾌적', '신기함/관심', '아껴주는', '부끄러움', '즐거움/신남', '깨달음', '흐뭇함(귀여움/예쁨)', '행복', '기쁨', '안심/신뢰'}
negative_labels = {'화남/분노', '안타까움/실망', '의심/불신', '한심함', '역겨움/징그러움', '짜증', '어이없음', '증오/혐오', '경악', '재미없음', '불평/불만', '지긋지긋'}
neutral_labels = {'슬픔', '우쭐댐/무시함', '공포/무서움', '절망', '패배/자기혐오', '귀찮음', '힘듦/지침', '죄책감', '당황/난처', '부담/안내킴', '서러움', '불쌍함/연민', '놀람', '불안/걱정', '없음'}

df = pd.DataFrame(data)

# 라벨 문자열을 숫자 리스트로 변환
def parse_labels(label_string):
    return list(map(int, label_string.strip('[]').split()))

# 라벨 인덱스를 이름으로 변환 후, 감정 분류
def classify_sentiment(row):
    label_string = row['labels']
    labels = parse_labels(label_string)  # 라벨 문자열을 리스트로 변환
    label_names = [LABELS[i] for i in labels]  # 라벨 인덱스로 라벨 이름 가져오기
    
    positive_count = sum(label in positive_labels for label in label_names)
    negative_count = sum(label in negative_labels for label in label_names)
    neutral_count = sum(label in neutral_labels for label in label_names)
    
    # 최종 라벨 결정
    if positive_count > negative_count:
        return '긍정'
    elif negative_count > positive_count:
        return '부정'
    else:
        return '중립'

# 새로운 라벨 열 추가
df['final_label'] = df.apply(classify_sentiment, axis=1)
print(df[['ID', 'text', 'final_label']])

          ID                                               text final_label
0      39087              내가 톰행크스를 좋아하긴 했나보다... 초기 영화 빼고는 다 봤네.          긍정
1      30893  정말 상상을 초월하는 무개념 진상들 상대하다 우울증, 공항장애 걸리는 공무원 많아요...          부정
2      45278  새로운 세상과 조우한 자의 어린아이 같은 반응, 어쩌면 회복된 것은 눈이 아닌 순수...          긍정
3      16398       미역은 원생생물계 산호초는 동물ㅇㅇ 아 미역이 바다의 새ㄱㅇㄱㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ          긍정
4      13653                        네 맞습니다 플스는 역시 30프레임이 어울리죠 ㅎ          긍정
...      ...                                                ...         ...
49995   6161  일단 내용만 보고놓자면 크게 신경쓸일은 아닌거같네요 한국룸메랑 같이 비교하지마세요 ...          긍정
49996  21130  캐릭터가 프린트된 제품인줄 알았는데 스티커임 손으로 조물조물 빨았는데 캐릭터들이 쪼...          부정
49997   6788                          마지막에 마카롱으로 미키마우스만든거 넘귀엽다ㅋ          긍정
49998  45946                    너무 멋진 친구를 두셨네요  작가님 함께 같이 살아갑시다          긍정
49999   9159                          강려크하시니 오늘은 검색하지 않겟습니다. ㅋㅋ          긍정

[50000 rows x 3 columns]


In [20]:
print(len(df[df['final_label']=='긍정']))
print(len(df[df['final_label']=='부정']))
print(len(df[df['final_label']=='중립']))

22542
25172
2286


In [25]:
df.to_csv('data/kote_relabeled.csv')